In [1]:
# Importing necessary libraries
import nltk
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import DoubleType, ArrayType, StringType
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, udf
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql.types import DoubleType, ArrayType, StringType
from pyspark.sql.types import DoubleType
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import explode
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout
from pyspark.sql import functions as F

[nltk_data] Downloading package punkt to /home/hduser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hduser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Importing PySpark
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read CSV") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

24/04/04 18:33:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
# Read the CSV file into a DataFrame
data_df = spark.read.load("hdfs://localhost:9000/user1/train.csv", format="csv", header=True)

In [5]:
data_df.show(3)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
+-----------+--------------------+--------------------+
only showing top 3 rows



In [6]:
# Check the number of rows
num_rows = data_df.count()
print("Number of rows:", num_rows)

# Check the list of column names
column_names = data_df.columns
print("Column names:", column_names)


Number of rows: 3000000
Column names: ['class_index', 'review_title', 'review_text']


In [7]:
#Printing Schema
print("Schema:")
data_df.printSchema()

Schema:
root
 |-- class_index: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_text: string (nullable = true)



In [8]:
# Show sample rows after removing null values
data_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          4|    Chrono Cross OST|"The music of Yas...|
|          5| Too good to be true|Probably the grea...|
|          5|There's a reason ...|There's a reason ...|
|          1|        Buyer beware|"This is a self-p...|
|          4|Errors, but great...|I was a dissapoin...|
|          1|          The Worst!|A complete waste ...|
|          1|           Oh please|I guess you have ...|
|          1|Awful beyond belief!|"I feel I have to...|
|          4|A romantic zen ba...|"When you hear fo...|
|          5|Lower leg comfort...|Excellent stockin...|
|          3|Delivery was very...|It took almost 3 ...|
|          2|sizes recomended ...|sizes are much

In [9]:
# Group by 'class_index' column and perform aggregation
aggregated_df = data_df.groupBy("class_index").agg(
    F.count("*").alias("count_reviews"),
    F.avg("class_index").alias("avg_class_index")
)

In [10]:
# Show the aggregated DataFrame
aggregated_df.show()

+-----------+-------------+---------------+
|class_index|count_reviews|avg_class_index|
+-----------+-------------+---------------+
|          3|       600000|            3.0|
|          5|       600000|            5.0|
|          1|       600000|            1.0|
|          4|       600000|            4.0|
|          2|       600000|            2.0|
+-----------+-------------+---------------+



In [11]:
# Sorting the DataFrame by a single column
sorted_df = data_df.orderBy("class_index")

In [12]:
# Showing the sorted DataFrame
sorted_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          1|  Read Dante Instead|Reading this book...|
|          1|         Didn't work|This tape was imm...|
|          1|        unhappy feet|A disappointment ...|
|          1|                Poor|"This is actually...|
|          1|Really? Magic gecko?|Right about the t...|
|          1|             Useless|I don't think it ...|
|          1|             Bad Buy|I bought this rin...|
|          1|  Poor Sound Quality|The sound quality...|
|          1|    Frustrating game|The good news is ...|
|          1|     Waste of money.|This thing is a p...|
|          1|     A Steaming Pile|"This album is ho...|
|          1|          Wrong item|Ordered the Belki...|
|          1|     TOTALLY USELESS|"Not only is the ...|
|          1|Product did not work|This didn't work ...|
|          1|Statistics for st...|This book will

In [13]:
# Sort in descending order, using desc() function
sorted_desc_df = data_df.orderBy(data_df["class_index"].desc())
sorted_desc_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          5|For those of us w...|"Louis adventures...|
|          5|   Coolest game eva!|I love the nancy ...|
|          5|    Soft and Cuddly!|My son loves his ...|
|          5|      Kitchen Review|My husband and I ...|
|          5|      The New Empire|From the other st...|
|          5|Kind of creepy bu...|"In the begining ...|
|          5|A Reference Recor...|In the mid '70s, ...|
|          5|great product at ...|I have purchased ...|
|          5|           excellent|Exactly what I no...|
|          5|          Great pan!|This is a great q...|
|          5|Everything a roma...|Just finished thi...|
|          5|this is the one y...|best value in a t...|
|          5|Ice cream scoop, ...|Has been operatin...|
|          5|  Norpro popover pan|We had popovers b...|
|          5|       giving it all|Real life lyri

In [14]:
# Sorting by multiple columns
multi_sorted_df = data_df.orderBy("class_index", "review_title")
multi_sorted_df.show()

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          1|                null|"Unless you're a ...|
|          1|                null|"I was extremely ...|
|          1|                null|Just wanted to vo...|
|          1|                null|Can't write a rev...|
|          1|                null|These guys sure h...|
|          1|                null|I didn't really c...|
|          1|                null|"Some good rockin...|
|          1|                null|This is lightweig...|
|          1|                null|"I remember order...|
|          1|                null|And here we have ...|
|          1|                null|I ordered a 12 fo...|
|          1|                null|The battery was p...|
|          1|                   !|"This book is com...|
|          1|! - STAY AWAY! WO...|Please for the lo...|
|          1|! CAUTION - NOT T...|!! CAUTION!!Re

In [15]:
# Filtering rows where class_index is equal to 5
filtered_df = data_df.filter(data_df["class_index"] == 5)

In [16]:
# print filtered data
print("Filtered Data:")
filtered_df.show()

Filtered Data:
+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          5| Too good to be true|Probably the grea...|
|          5|There's a reason ...|There's a reason ...|
|          5|Lower leg comfort...|Excellent stockin...|
|          5|CHECK OUT THE K60...|Greetings. Kodak ...|
|          5|                ahhh|These will more t...|
|          5|   Excellent Product|This is an excell...|
|          5|"To ""Disappointe...|"You said ""...bu...|
|          5|Awesome - wanna u...|It's just awesome...|
|          5|"Very Happy - Has...|"I got this machi...|
|          5|This is a very go...|Many useful conce...|
|          5|Great reference b...|It seems somebody...|
|          5|Here is a little ...|After you watch a...|
|          5|Rochelle explains...

In [17]:
data_df.show(3)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
+-----------+--------------------+--------------------+
only showing top 3 rows



In [18]:
from pyspark.sql.functions import col, count

# Count null values in each column
null_counts = data_df.agg(*[count(col(c).isNull().cast("int")).alias(c) for c in data_df.columns])

# Show the null counts
null_counts.show()


+-----------+------------+-----------+
|class_index|review_title|review_text|
+-----------+------------+-----------+
|    3000000|     3000000|    3000000|
+-----------+------------+-----------+



In [19]:
# Show a sample of the DataFrame
data_df.show(5)

+-----------+--------------------+--------------------+
|class_index|        review_title|         review_text|
+-----------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|
|          5|           Inspiring|I hope a lot of p...|
|          5|The best soundtra...|I'm reading a lot...|
|          4|    Chrono Cross OST|"The music of Yas...|
|          5| Too good to be true|Probably the grea...|
+-----------+--------------------+--------------------+
only showing top 5 rows



In [20]:
from pyspark.sql.functions import col

# Filter for rows where review_title or review_text is empty
empty_title_df = data_df.filter(col("review_title") == "")
empty_text_df = data_df.filter(col("review_text") == "")

# Count the number of rows with empty review_title and review_text
empty_title_count = empty_title_df.count()
empty_text_count = empty_text_df.count()

print("Number of rows with empty review_title:", empty_title_count)
print("Number of rows with empty review_text:", empty_text_count)


Number of rows with empty review_title: 0
Number of rows with empty review_text: 0


In [21]:
from pyspark.sql.functions import concat_ws

# Concatenate review_title and review_text into a new column named text
data_df = data_df.withColumn('text', concat_ws(' ', data_df['review_title'], data_df['review_text']))
data_df.show()

+-----------+--------------------+--------------------+--------------------+
|class_index|        review_title|         review_text|                text|
+-----------+--------------------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|more like funchuc...|
|          5|           Inspiring|I hope a lot of p...|Inspiring I hope ...|
|          5|The best soundtra...|I'm reading a lot...|The best soundtra...|
|          4|    Chrono Cross OST|"The music of Yas...|Chrono Cross OST ...|
|          5| Too good to be true|Probably the grea...|Too good to be tr...|
|          5|There's a reason ...|There's a reason ...|There's a reason ...|
|          1|        Buyer beware|"This is a self-p...|Buyer beware "Thi...|
|          4|Errors, but great...|I was a dissapoin...|Errors, but great...|
|          1|          The Worst!|A complete waste ...|The Worst! A comp...|
|          1|           Oh please|I guess you have ...|Oh please I guess...|

In [22]:
from pyspark.sql.functions import concat, concat_ws, col

# Concatenate review_title and review_text into a new column named text
data_df = data_df.withColumn('text', concat_ws(' ', col('review_title'), col('review_text')))

data_df.show(5)

+-----------+--------------------+--------------------+--------------------+
|class_index|        review_title|         review_text|                text|
+-----------+--------------------+--------------------+--------------------+
|          3|  more like funchuck|"Gave this to my ...|more like funchuc...|
|          5|           Inspiring|I hope a lot of p...|Inspiring I hope ...|
|          5|The best soundtra...|I'm reading a lot...|The best soundtra...|
|          4|    Chrono Cross OST|"The music of Yas...|Chrono Cross OST ...|
|          5| Too good to be true|Probably the grea...|Too good to be tr...|
+-----------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [23]:
from pyspark.sql.functions import col

# Drop the review_title and review_text columns
data_df = data_df.drop(col('review_title')).drop(col('review_text'))

# Show the resulting DataFrame
data_df.show()


+-----------+--------------------+
|class_index|                text|
+-----------+--------------------+
|          3|more like funchuc...|
|          5|Inspiring I hope ...|
|          5|The best soundtra...|
|          4|Chrono Cross OST ...|
|          5|Too good to be tr...|
|          5|There's a reason ...|
|          1|Buyer beware "Thi...|
|          4|Errors, but great...|
|          1|The Worst! A comp...|
|          1|Oh please I guess...|
|          1|Awful beyond beli...|
|          4|A romantic zen ba...|
|          5|Lower leg comfort...|
|          3|Delivery was very...|
|          2|sizes recomended ...|
|          3|Overbury Full of ...|
|          1|Another Abysmal D...|
|          4|Wardell's book is...|
|          4|i liked this albu...|
|          3|Better than I tho...|
+-----------+--------------------+
only showing top 20 rows



In [24]:
# Tokenization
tokenize_udf = udf(lambda text: word_tokenize(text), ArrayType(StringType()))

In [25]:
tokenized_df = data_df.withColumn("tokens", tokenize_udf(col("text")))
tokenized_df.show()

+-----------+--------------------+--------------------+
|class_index|                text|              tokens|
+-----------+--------------------+--------------------+
|          3|more like funchuc...|[more, like, func...|
|          5|Inspiring I hope ...|[Inspiring, I, ho...|
|          5|The best soundtra...|[The, best, sound...|
|          4|Chrono Cross OST ...|[Chrono, Cross, O...|
|          5|Too good to be tr...|[Too, good, to, b...|
|          5|There's a reason ...|[There, 's, a, re...|
|          1|Buyer beware "Thi...|[Buyer, beware, `...|
|          4|Errors, but great...|[Errors, ,, but, ...|
|          1|The Worst! A comp...|[The, Worst, !, A...|
|          1|Oh please I guess...|[Oh, please, I, g...|
|          1|Awful beyond beli...|[Awful, beyond, b...|
|          4|A romantic zen ba...|[A, romantic, zen...|
|          5|Lower leg comfort...|[Lower, leg, comf...|
|          3|Delivery was very...|[Delivery, was, v...|
|          2|sizes recomended ...|[sizes, recome

In [26]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
remove_stopwords_udf = udf(lambda tokens: [token for token in tokens if token not in stop_words], ArrayType(StringType()))

In [27]:
filtered_df = tokenized_df.withColumn("filtered_tokens", remove_stopwords_udf(col("tokens")))
filtered_df.show()

+-----------+--------------------+--------------------+--------------------+
|class_index|                text|              tokens|     filtered_tokens|
+-----------+--------------------+--------------------+--------------------+
|          3|more like funchuc...|[more, like, func...|[like, funchuck, ...|
|          5|Inspiring I hope ...|[Inspiring, I, ho...|[Inspiring, I, ho...|
|          5|The best soundtra...|[The, best, sound...|[The, best, sound...|
|          4|Chrono Cross OST ...|[Chrono, Cross, O...|[Chrono, Cross, O...|
|          5|Too good to be tr...|[Too, good, to, b...|[Too, good, true,...|
|          5|There's a reason ...|[There, 's, a, re...|[There, 's, reaso...|
|          1|Buyer beware "Thi...|[Buyer, beware, `...|[Buyer, beware, `...|
|          4|Errors, but great...|[Errors, ,, but, ...|[Errors, ,, great...|
|          1|The Worst! A comp...|[The, Worst, !, A...|[The, Worst, !, A...|
|          1|Oh please I guess...|[Oh, please, I, g...|[Oh, please, I, g...|

In [28]:
# Sentiment Analysis using NLTK
sia = SentimentIntensityAnalyzer()
sentiment_udf = udf(lambda text: sia.polarity_scores(text)["compound"], DoubleType())

In [29]:
sentiment_df = filtered_df.withColumn("sentiment_score", sentiment_udf(col("text")))
sentiment_df.show()

+-----------+--------------------+--------------------+--------------------+---------------+
|class_index|                text|              tokens|     filtered_tokens|sentiment_score|
+-----------+--------------------+--------------------+--------------------+---------------+
|          3|more like funchuc...|[more, like, func...|[like, funchuck, ...|         0.5095|
|          5|Inspiring I hope ...|[Inspiring, I, ho...|[Inspiring, I, ho...|         0.9822|
|          5|The best soundtra...|[The, best, sound...|[The, best, sound...|         0.8957|
|          4|Chrono Cross OST ...|[Chrono, Cross, O...|[Chrono, Cross, O...|          0.926|
|          5|Too good to be tr...|[Too, good, to, b...|[Too, good, true,...|         0.9807|
|          5|There's a reason ...|[There, 's, a, re...|[There, 's, reaso...|         0.6369|
|          1|Buyer beware "Thi...|[Buyer, beware, `...|[Buyer, beware, `...|        -0.6479|
|          4|Errors, but great...|[Errors, ,, but, ...|[Errors, ,, gre

In [30]:
# Tokenize the text
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
tokenized_df = tokenizer.transform(data_df)

In [31]:
# Remove stopwords
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
filtered_df = remover.transform(tokenized_df)

In [32]:
# Explode the filtered tokens
exploded_df = filtered_df.withColumn("token", explode("filtered_tokens"))

In [33]:
sentiment_df.show(3)

+-----------+--------------------+--------------------+--------------------+---------------+
|class_index|                text|              tokens|     filtered_tokens|sentiment_score|
+-----------+--------------------+--------------------+--------------------+---------------+
|          3|more like funchuc...|[more, like, func...|[like, funchuck, ...|         0.5095|
|          5|Inspiring I hope ...|[Inspiring, I, ho...|[Inspiring, I, ho...|         0.9822|
|          5|The best soundtra...|[The, best, sound...|[The, best, sound...|         0.8957|
+-----------+--------------------+--------------------+--------------------+---------------+
only showing top 3 rows



In [34]:
# Compute vocabulary size
vocabulary_size = exploded_df.select("token").distinct().count()
print("Vocabulary Size:", vocabulary_size)

Vocabulary Size: 3440123


In [35]:
# Convert Spark DataFrame to Pandas DataFrame
X = sentiment_df.select("sentiment_score").toPandas()
y = sentiment_df.select("class_index").toPandas()

----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 48570)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent cal

In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Convert Pandas DataFrames to NumPy arrays and ensure correct data types
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('int32')  # Ensure integer labels
y_test = y_test.astype('int32')

In [38]:
vocabulary_size = 1000  # Adjust as needed

In [39]:
X_train = X_train[X_train < vocabulary_size]
X_test = X_test[X_test < vocabulary_size]

In [40]:
import tensorflow as tf

# Set GPU memory growth to avoid allocating all GPU memory upfront
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [41]:
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=128))  
model.add(Conv1D(filters=128, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [42]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [44]:
epochs = 30  # Adjust as needed

# Optimize batch size
batch_size = 64 

In [45]:
# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 56s 1ms/step - accuracy: 0.2003 - loss: -737936920281088.0000 - val_accuracy: 0.2004 - val_loss: -19344659402719232.0000
Epoch 2/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.1994 - loss: -63422648188665856.0000 - val_accuracy: 0.2004 - val_loss: -392202807014326272.0000
Epoch 3/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 56s 1ms/step - accuracy: 0.2005 - loss: -719899603289243648.0000 - val_accuracy: 0.2004 - val_loss: -2444951296214564864.0000
Epoch 4/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - accuracy: 0.1999 - loss: -3673536792998445056.0000 - val_accuracy: 0.2004 - val_loss: -9195827621311545344.0000
Epoch 5/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 82s 1ms/step - accuracy: 0.1996 - loss: -12490076058558660608.0000 - val_accuracy: 0.2004 - val_loss: -26039406226153930752.0000
Epoch 6/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - accuracy: 0.2004 - loss: -33251969407899402240.0000 - val_accuracy: 0.2004 - val_loss: -614198

2024-04-04 20:05:36.683647: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 19200000 bytes after encountering the first element of size 19200000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


37500/37500 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.2004 - loss: -76330555321181534158848.0000 - val_accuracy: 0.2004 - val_loss: -87596769156210821693440.0000
Epoch 28/30
  105/37500 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.2002 - loss: -87135573532770318680064.0000

2024-04-04 20:06:32.026540: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 19200000 bytes after encountering the first element of size 19200000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


37500/37500 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.1997 - loss: -91692964154090127884288.0000 - val_accuracy: 0.2004 - val_loss: -104564386997103826567168.0000
Epoch 29/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.1998 - loss: -109270711658100778074112.0000 - val_accuracy: 0.2004 - val_loss: -124054876313253509595136.0000
Epoch 30/30
37500/37500 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - accuracy: 0.1993 - loss: -129496071339847014416384.0000 - val_accuracy: 0.2004 - val_loss: -146352351190752165888000.0000


In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


18750/18750 ━━━━━━━━━━━━━━━━━━━━ 7s 360us/step - accuracy: 0.2005 - loss: -146261918910234566328320.0000
Test Loss: -1.4634928874300555e+23
Test Accuracy: 0.20043833553791046


In [47]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print out model predictions
y_pred = model.predict(X_test)
print("Predictions:", y_pred)

# Print out confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


18750/18750 ━━━━━━━━━━━━━━━━━━━━ 7s 366us/step - accuracy: 0.2005 - loss: -146261918910234566328320.0000
Test Loss: -1.4634928874300555e+23
Test Accuracy: 0.20043833553791046
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 7s 363us/step
Predictions: [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
Confusion Matrix:
[[120263      0      0      0      0]
 [119812      0      0      0      0]
 [119896      0      0      0      0]
 [119849      0      0      0      0]
 [120180      0      0      0      0]]
